Gerf-Yolo works now, so now sleap data against gerf

---

First we're going to download ultralytics repository which contains the yolo repository.

In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 76933, done.
remote: Counting objects: 100% (787/787), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 76933 (delta 691), reused 542 (delta 531), pack-reused 76146 (from 3)
Receiving objects: 100% (76933/76933), 41.03 MiB | 26.06 MiB/s, done.
Resolving deltas: 100% (57819/57819), done.


We're going to install the SLEAP libraries for GPU
If you want CPU then comment command out and uncomment second command
WARNING: When installing this package, it will delete your current version of CUDA. Run in COLAB or separate environment (conda)

In [2]:
!# CUDA 12.8
!pip install "sleap[nn]" --extra-index-url https://download.pytorch.org/whl/cu128 --index-url https://pypi.org/simple
#CPU Command:
#!pip install "sleap[nn]" --extra-index-url https://download.pytorch.org/whl/cpu --index-url https://pypi.org/simple

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu128
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 85.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.3/314.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 911.5/911.5 kB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.8/557.8 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Let's verify that SLEAP module works

In [3]:
!python -c "import sleap_io as sio; print(f'SLEAP module loaded.\nsleap_io version: {sio.__version__}')"

SLEAP module loaded.
sleap_io version: 0.5.7


We're going to unzip the environment setup which has all the files we need to run for GERF Yolo and SLEAP API conversion

In [4]:
!unzip environmentSetup.zip

Archive:  environmentSetup.zip
  inflating: custom/__init__.py      
  inflating: custom/gerf_bra.py      
  inflating: custom/mouse-pose.yaml  
  inflating: custom/tasks.py         
  inflating: custom/yolo11-gerfpose.yaml  
  inflating: inferenceGERF.py        
  inflating: mainGERF.py             
  inflating: sleapVideoFrames.py     


Now we're going to download the SLEAP packages for Validation and Test
Datasets correspond to the following section: https://docs.sleap.ai/v1.5.1/reference/datasets/#mice_hc

In [5]:
# Download with progress bar and resume capability
!mkdir -p datasets
!wget -P datasets/ -c --show-progress https://storage.googleapis.com/sleap-data/datasets/eleni_mice/random_split1/train.pkg.slp
!echo "Done downloading training package"

!wget -P datasets/ -c --show-progress https://storage.googleapis.com/sleap-data/datasets/eleni_mice/random_split1/val.pkg.slp
!echo "Done downloading validation package"

--2025-12-08 00:58:43--  https://storage.googleapis.com/sleap-data/datasets/eleni_mice/random_split1/train.pkg.slp
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 696486384 (664M) [application/octet-stream]
Saving to: ‘datasets/train.pkg.slp’

train.pkg.slp       100%[===================>] 664.22M  64.5MB/s    in 11s     

2025-12-08 00:58:54 (62.5 MB/s) - ‘datasets/train.pkg.slp’ saved [696486384/696486384]

Done downloading training package
--2025-12-08 00:58:54--  https://storage.googleapis.com/sleap-data/datasets/eleni_mice/random_split1/val.pkg.slp
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 142.250.141.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent

Next, we're going to copy over the custom files to the ultralytics repository
Note, the yolo11_custom.yaml file contains the architecture that incorporates GERF model. We'll leave it inside the custom directory on purpose.

In [6]:
!cp custom/__init__.py ultralytics/ultralytics/nn/modules
!cp custom/gerf_bra.py ultralytics/ultralytics/nn/modules
!cp custom/tasks.py ultralytics/ultralytics/nn

Now, we need to export the python path to include this repository to use with our python code.
Ideally, we would install with "pip install -e ." and the ultralytics toml file but for some reason,
it would do a clean install and ignore my changes
1. First we'll collect your current path
2. Then we'll check what's on PYTHONPATH
3. Add the outputs of your current path from 1 and the PYTHONPATH to the environment variable
4. Finally, we'll verify the variable was set

In [7]:
!echo "Check where the ultralytics repo is stored"
!pwd

Check where the ultralytics repo is stored
/content


In [8]:
!echo "Check current python path. Should be empty at every new session"
!echo $PYTHONPATH

Check current python path. Should be empty at every new session
/env/python


In [9]:
!echo "Now let's set the path. I got the below path from the pwd command and added in ultralytics"
%env PYTHONPATH=/content/ultralytics:/env/python

Now let's set the path. I got the below path from the pwd command and added in ultralytics
env: PYTHONPATH=/content/ultralytics:/env/python


In [10]:
!echo "Check current python path. Should be updated now"
!echo $PYTHONPATH

Check current python path. Should be updated now
/content/ultralytics:/env/python


Now we can test out if the GERF additions are ready to use
The first cell tests the CLI
The second cell actually imports the custom yolo repository for python notebook usage

In [11]:
!python -c "from ultralytics import YOLO; from ultralytics.nn.modules import GERF_BRA_Simple; print('All good!')"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
All good!


Now we'll convert the SLEAP Mouse training and validation files to a COCO representative format

In [18]:
!python sleapVideoFrames.py

Found 1178 labeled frames
Videos: 40
Skeleton: 5 keypoints
Keypoints: ['snout', 'earL', 'earR', 'tb', 'tt']
  Processed 100 images, 200 annotations
  Processed 200 images, 400 annotations
  Processed 300 images, 600 annotations
  Processed 400 images, 800 annotations
  Processed 500 images, 1000 annotations
  Processed 600 images, 1200 annotations
  Processed 700 images, 1400 annotations
  Processed 800 images, 1600 annotations
  Processed 900 images, 1800 annotations
  Processed 1000 images, 2000 annotations
  Processed 1100 images, 2200 annotations

Conversion complete!
  Images: 1178
  Annotations: 2356
  Average annotations per image: 2.00
  Saved to: datasets/mouse-pose/images/train
  Keypoints: 5
  Names: ['snout', 'earL', 'earR', 'tb', 'tt']
1. Create YAML config:
   nc: 1
   kpt_shape: [5, 3]
   names: ['mouse']

2. Keypoint names: ['snout', 'earL', 'earR', 'tb', 'tt']
Found 148 labeled frames
Videos: 35
Skeleton: 5 keypoints
Keypoints: ['snout', 'earL', 'earR', 'tb', 'tt']
  P

Now convert COCO to YOLO format

In [19]:
!python sleapAnnotations.py

Converting training set...
Converting 1178 images...
  Converted 100/1178 images
  Converted 200/1178 images
  Converted 300/1178 images
  Converted 400/1178 images
  Converted 500/1178 images
  Converted 600/1178 images
  Converted 700/1178 images
  Converted 800/1178 images
  Converted 900/1178 images
  Converted 1000/1178 images
  Converted 1100/1178 images
Conversion complete!
   Converted 1178 label files
   Output: datasets/mouse-pose/labels/train

Converting validation set...
Converting 148 images...
  Converted 100/148 images
Conversion complete!
   Converted 148 label files
   Output: datasets/mouse-pose/labels/val

Dataset is now ready for YOLO training


Train the GERF Model against the SLEAP data

In [22]:
!python mainGERF.py
!echo "Done training"

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Training on GPU
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom/mouse-pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom/yolo11-gerfpose.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11gerf_mouse_gpu, nbs=64, nms=False, opset=None, optimize

In [23]:
!python inferenceGERF.py
!mkdir -p results
!mkdir -p results/gerf
!mv *.jpg results/gerf

Running GERF
Found 148 total images in datasets/mouse-pose/images/val
Running inference on first 50 images
Processing [1/50]: img_00000000.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000000.jpg: 512x640 2 mouses, 60.6ms
Speed: 2.9ms preprocess, 60.6ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)
Processing [2/50]: img_00000001.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000001.jpg: 512x640 3 mouses, 9.9ms
Speed: 2.0ms preprocess, 9.9ms inference, 2.1ms postprocess per image at shape (1, 3, 512, 640)
Processing [3/50]: img_00000002.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000002.jpg: 512x640 1 mouse, 9.3ms
Speed: 1.8ms preprocess, 9.3ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
Processing [4/50]: img_00000003.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000003.jpg: 512x640 (no detections), 9.0ms
Speed: 1.8ms preprocess, 9.0ms inference, 0.5ms postprocess per image at shape (1

In [25]:
!python mainYolo11.py

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=custom/mouse-pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=custom/yolo11-pose.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_mouse_gpu, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto,

In [27]:
!python inferenceYOLO.py
!mkdir -p results
!mkdir -p results/yolo
!mv *.jpg results/yolo

Running Normal Yolo
Found 148 total images in datasets/mouse-pose/images/val
Running inference on first 50 images
Processing [1/50]: img_00000000.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000000.jpg: 512x640 2 mouses, 58.1ms
Speed: 2.7ms preprocess, 58.1ms inference, 2.6ms postprocess per image at shape (1, 3, 512, 640)
Processing [2/50]: img_00000001.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000001.jpg: 512x640 2 mouses, 8.6ms
Speed: 1.9ms preprocess, 8.6ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
Processing [3/50]: img_00000002.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000002.jpg: 512x640 3 mouses, 8.1ms
Speed: 1.8ms preprocess, 8.1ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 640)
Processing [4/50]: img_00000003.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000003.jpg: 512x640 2 mouses, 7.8ms
Speed: 1.8ms preprocess, 7.8ms inference, 1.9ms postprocess per image at shape (

Attempt to Annotate skeleton with new plotting mechanism

In [30]:
!cp custom/plotting_mouse.py ultralytics/ultralytics/utils/plotting.py

Now run inferance again

In [32]:
!python inferenceGERF.py
!mkdir -p results_annotate
!mkdir -p results_annotate/gerf
!mv *.jpg results_annotate/gerf

Running GERF
Found 148 total images in datasets/mouse-pose/images/val
Running inference on first 50 images
Processing [1/50]: img_00000000.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000000.jpg: 512x640 2 mouses, 58.5ms
Speed: 2.6ms preprocess, 58.5ms inference, 2.2ms postprocess per image at shape (1, 3, 512, 640)
Processing [2/50]: img_00000001.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000001.jpg: 512x640 3 mouses, 11.1ms
Speed: 2.1ms preprocess, 11.1ms inference, 2.1ms postprocess per image at shape (1, 3, 512, 640)
Processing [3/50]: img_00000002.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000002.jpg: 512x640 1 mouse, 9.5ms
Speed: 1.8ms preprocess, 9.5ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 640)
Processing [4/50]: img_00000003.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000003.jpg: 512x640 (no detections), 9.8ms
Speed: 1.8ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape 

In [33]:
!python inferenceYOLO.py
!mkdir -p results_annotate
!mkdir -p results_annotate/yolo
!mv *.jpg results_annotate/yolo

Running Normal Yolo
Found 148 total images in datasets/mouse-pose/images/val
Running inference on first 50 images
Processing [1/50]: img_00000000.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000000.jpg: 512x640 2 mouses, 83.6ms
Speed: 3.6ms preprocess, 83.6ms inference, 3.4ms postprocess per image at shape (1, 3, 512, 640)
Processing [2/50]: img_00000001.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000001.jpg: 512x640 2 mouses, 10.8ms
Speed: 2.3ms preprocess, 10.8ms inference, 2.4ms postprocess per image at shape (1, 3, 512, 640)
Processing [3/50]: img_00000002.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000002.jpg: 512x640 3 mouses, 10.3ms
Speed: 2.2ms preprocess, 10.3ms inference, 2.9ms postprocess per image at shape (1, 3, 512, 640)
Processing [4/50]: img_00000003.jpg

image 1/1 /content/datasets/mouse-pose/images/val/img_00000003.jpg: 512x640 2 mouses, 10.1ms
Speed: 2.3ms preprocess, 10.1ms inference, 2.3ms postprocess per image at s

Reset plotting to normal just in case

In [34]:
!cp custom/plotting_normal.py ultralytics/ultralytics/utils/plotting.py